In [ ]:
import os
import re

import numpy as np
from matplotlib import pyplot as plt
from scipy import signal
from scipy.io import wavfile
from pydub import AudioSegment

import simfile
from simfile.notes import NoteData, NoteType
from simfile.timing import Beat, TimingData
from simfile.timing.engine import TimingEngine

In [ ]:
test_simfile_dir = r"C:\Games\ITGmania\Songs\ITGAlex's Compilation 4\Fracture Ray"

test_audio_path = None
test_simfile_path = None
for f in os.listdir(test_simfile_dir):
    if os.path.splitext(f)[1] in ['.ssc', '.sm']:
        if (test_simfile_path is None) or (os.path.splitext(test_simfile_path)[1] == '.sm'):
            test_simfile_path = os.path.join(test_simfile_dir, f)
print(test_simfile_path)

test_simfile = simfile.open(test_simfile_path)
chart = test_simfile.charts[0]
if not hasattr(chart, 'music') or chart.music is None:
    test_audio_path = os.path.join(test_simfile_dir, test_simfile.music)
else:
    test_audio_path = os.path.join(test_simfile_dir, chart.music)
print(test_audio_path)

engine = TimingEngine(TimingData(test_simfile, chart))

In [ ]:
audio_ext = os.path.splitext(test_audio_path)[1]
audio = AudioSegment.from_file(test_audio_path, format=audio_ext[1:])
audio_data = np.array(audio.get_array_of_samples())

# https://stackoverflow.com/questions/53633177/how-to-read-a-mp3-audio-file-into-a-numpy-array-save-a-numpy-array-to-mp3
if audio.channels == 2:
    audio_data = audio_data.reshape((-1, 2))
audio_data = audio_data / 2**15

In [ ]:
print(audio_data.shape)
print(audio.frame_rate)

# https://stackoverflow.com/questions/44787437/how-to-convert-a-wav-file-to-a-spectrogram-in-python3
# https://stackoverflow.com/questions/47954034/plotting-spectrogram-in-audio-analysis
window_ms = 5
step_ms = 1
eps = 1e-9

nperseg = int(audio.frame_rate * window_ms * 1e-3)
noverlap = nperseg - int(audio.frame_rate * step_ms * 1e-3)
frequencies, times, spectrogram = signal.spectrogram(
    audio_data[:, 0],
    fs=audio.frame_rate,
    window='hann',
    nperseg=nperseg,
    noverlap=noverlap,
    detrend=False
)
# frequencies, times, spectrogram = signal.spectrogram(a, fs=1000)
# print(spectrogram[:5, :5])
splog = np.log2(spectrogram + eps)
fig = plt.figure(figsize=(30, 6))
plt.pcolormesh(times, frequencies, splog)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
fingerprint_sec = 0.10
actual_step = (nperseg - noverlap) / audio.frame_rate
fingerprint_size = 2 * int(0.5 * round(fingerprint_sec / actual_step))
fingerprint_times = np.arange(-fingerprint_size // 2, fingerprint_size // 2) * actual_step
frequency_emphasis_factor = 3000 # None

# print(audio_data.shape)
# print(frequencies.shape)
# print(spectrogram.shape)
# print(nperseg)
# print(times[:5])
# print(actual_step)
# print(fingerprint_size)

b = 0
acc = np.zeros((frequencies.size, fingerprint_size))
digest = np.zeros((0, fingerprint_size))
while True:
    t = engine.time_at(b)
    b += 1
    if (t < 0):
        continue
    if (t > audio.duration_seconds):
        break

    t_s = max(0,                   int(t / actual_step - fingerprint_size * 0.5))
    t_f = min(audio_data.shape[0], int(t / actual_step + fingerprint_size * 0.5))
    if (t_f - t_s != fingerprint_size):
        # Not enough data at this beat tbh
        continue
    
    # print(f'{t}: {t_s} -> {times[t_s]}, {t_f} -> {times[t_f]}')
    frequency_weights = 1
    if frequency_emphasis_factor is not None:
        frequency_weights = np.tile(frequencies * np.exp(-frequencies / frequency_emphasis_factor), [fingerprint_size, 1]).T
    spfilt = splog[:, t_s:t_f] * frequency_weights
    acc += spfilt
    digest = np.vstack([digest, np.sum(spfilt, axis=0)])

    if b in []:
        acc_log = splog[:, t_s:t_f]
        fig = plt.figure(figsize=(6, 6))
        plt.pcolormesh(fingerprint_times, frequencies, acc_log)
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        plt.show()
    


In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.pcolormesh(fingerprint_times, frequencies, acc)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

fig = plt.figure(figsize=(6, 6))
plt.pcolormesh(fingerprint_times, np.arange(digest.shape[0]), digest)
plt.ylabel('Beat Index')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
# Loudest point of attack
time_edge_kernel = [
    [1, 3, 10, 3, 1],
    [1, 3, 10, 3, 1],
    [1, 3, 10, 3, 1],
    [1, 3, 10, 3, 1],
    [1, 3, 10, 3, 1]
]
time_edge_offset = 0.000
if True:
    # Leading edge of attack
    time_edge_kernel = [
        [1, 3, 10, 0, -10, -3, -1],
        [1, 3, 10, 0, -10, -3, -1],
        [1, 3, 10, 0, -10, -3, -1],
        [1, 3, 10, 0, -10, -3, -1],
        [1, 3, 10, 0, -10, -3, -1]
    ]
    time_edge_offset = 0.002

acc_edge = signal.convolve2d(acc, time_edge_kernel, mode='same', boundary='wrap')
# acc_edge = signal.convolve2d(digest, time_edge_kernel, mode='same', boundary='wrap')
acc_edge_sum = np.sum(acc_edge, axis=0)
fingerprint_times = np.arange(-fingerprint_size // 2, fingerprint_size // 2) * actual_step
sync_bias = fingerprint_times[np.argmax(acc_edge_sum)] + time_edge_offset
if abs(sync_bias - 0.009) < abs(sync_bias):
    probable_bias = '+9ms'
else:
    probable_bias = 'null'

simfile_artist = test_simfile.artisttranslit or test_simfile.artist
simfile_title = test_simfile.titletranslit or test_simfile.title
plot_title = f'Sync fingerprint for {simfile_artist} - "{simfile_title}"\nDerived sync bias: {sync_bias:0.3f} (probably {probable_bias})'
time_ticks = np.arange(-fingerprint_sec * 1000, fingerprint_sec * 1000, 10)
print(time_ticks)

print(f'Sync bias: {sync_bias:0.3f}')

fig = plt.figure(figsize=(6, 6))
plt.pcolormesh(fingerprint_times * 1e3, frequencies, acc)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [msec]')
plt.xticks(time_ticks)
plt.plot(np.ones(np.shape(frequencies)) * sync_bias * 1e3, frequencies, 'r-')
plt.title(plot_title)
plt.show()
fig.savefig(f'bias-freqdomain-{simfile_title}.png')

fig = plt.figure(figsize=(6, 6))
plt.pcolormesh(fingerprint_times * 1e3, np.arange(digest.shape[0]), digest)
plt.ylabel('Beat Index')
plt.xlabel('Time [msec]')
plt.xticks(time_ticks)
plt.plot(np.ones(np.shape(digest)[0]) * sync_bias * 1e3, np.arange(digest.shape[0]), 'r-')
plt.title(plot_title)
plt.show()
fig.savefig(f'bias-beatdigest-{simfile_title}.png')

acc_edge_sum = np.sum(acc_edge, axis=0)
plt.plot(fingerprint_times, acc_edge_sum)
plt.show()

In [ ]:
pack_dir = r"C:\Games\ITGmania\Songs\ITGAlex's Compilation 4"
for d in os.listdir(pack_dir):
    if os.path.isdir(os.path.join(pack_dir, d)):
        print(d)


In [ ]:
a = 3.14159265
d = f'{a:0.3f}'
print(d)
b = '{:0.3f}'
c = b.format(a)
print(c)

In [ ]:
for test_simfile_path in [
    r'C:\Games\ITGmania\Songs\ephemera v0.2\crew -All Hands on the Deck-\crew.ssc',
    r'C:\Games\ITGmania\Songs\ephemera v0.2\PPBQ\ppbq.ssc',
    r'C:\Games\ITGmania\Songs\ephemera v0.2\Adrenalina\adrenalina.ssc'
]:
    base_simfile = simfile.open(test_simfile_path)
    for chart_index, chart in enumerate(base_simfile.charts):
        if any(k in chart for k in ['OFFSET', 'BPMS', 'STOPS', 'DELAYS', 'WARPS']):
            print(f'{base_simfile.title}: {chart_index} ({chart.difficulty}) has split timing')

    # for k, v in base_simfile.charts[1].items():
    #     print(f'{k}: {v}')

In [ ]:
a = [None, 3]
print(os.path.join(os.getcwd(), '*'))

In [ ]:
# Two aspects to confidence:
# - is the overall max response a clear winner, or are there other contenders?
#     - (second-highest - median) / (highest - median)
# - is the response outside of the max tight, or does it have a lot of variance/noise?
#     - stdev after scaling

import os
import csv
import glob
import numpy as np
from matplotlib import pyplot as plt

edge_discard = 3

conv_list = glob.glob(r'C:\Games\ITGmania\Songs\ITL Online 2023\__bias-check\convolution-*.csv')

conv_data = {}
conv_results = []

for i, f in enumerate(conv_list):
    t = []
    v = []
    with open(f, 'r', encoding='ascii') as fp:
        reader = csv.reader(fp)
        for row in reader:
            t += [float(row[0])]
            v += [float(row[1])]
    v_clip = v[edge_discard:-edge_discard]
    v_clip = np.interp(v_clip, (min(v_clip), max(v_clip)), (0, 1))
    t_clip = np.array(t[edge_discard:-edge_discard])
    v_std = np.std(v_clip)
    v_mean = np.mean(v_clip)
    v_median = np.median(v_clip)
    v_argmax = np.argmax(v_clip)
    v_max = v_clip[v_argmax]
    v_20 = np.percentile(v_clip, 20)
    v_80 = np.percentile(v_clip, 80)

    # Local maxima
    v_moving_diff = v_clip[1:] - v_clip[:-1]
    v_local_maxima = [i+1 for i, vmd in enumerate(zip(v_clip[:-2], v_clip[1:-1], v_clip[2:])) if (vmd[1] > vmd[0]) and (vmd[1] > vmd[2])]
    v_peaks = sorted(zip(v_local_maxima, v_clip[v_local_maxima]), key=lambda x: x[1], reverse=True)[:6]
    # print([f'{v[0]}: {v[1]}' for v in v_peaks])
    N_SAMPLES_NOT_NEAR = 10
    v_peaks_not_near = [v for v in v_peaks if abs(v[0] - v_peaks[0][0]) > N_SAMPLES_NOT_NEAR]
    maxness = 0
    if len(v_peaks_not_near) > 0:
        maxness = (v_peaks_not_near[0][1] - v_median) / (v_peaks[0][1] - v_median)

    # Another approach...
    THEORETICAL_UPPER = 0.83
    NEARNESS_SCALAR = 10    # milliseconds
    NEARNESS_OFFSET = 0.5   # milliseconds
    
    v_max_check = np.vstack((np.zeros_like(v_clip), (v_clip - v_median) / (v_max - v_median)))
    v_max_rivaling = np.max(v_max_check, axis=0)
    t_close_check = np.vstack((np.zeros_like(t_clip), abs(t_clip - t_clip[v_argmax]) - NEARNESS_OFFSET)) / NEARNESS_SCALAR
    t_close_enough = np.max(t_close_check, axis=0)
    max_influence = np.power(v_max_rivaling, 4) * np.power(t_close_enough, 1.5)
    total_max_influence = np.sum(max_influence) / np.size(max_influence)
    confidence = min(1, (1 - np.power(total_max_influence, 0.2)) / THEORETICAL_UPPER)

    print(f'{i:3d}/{len(conv_list):3d} {os.path.split(f)[1]:50s}: median = {v_median:0.6f}, stdev = {v_std:0.6f}, confidence = {confidence*100:0.2f}%')
    conv_results.append((os.path.split(f)[1], confidence))

    if confidence < 0.75 or i % 20 == 0:
        plt.figure(figsize=(6, 6))
        plt.title(os.path.split(f)[1] + f'\nstdev = {v_std:0.6f}, iqr = {v_80-v_20:0.6f}, confidence = {confidence*100:0.2f}%')
        # plt.plot(t[edge_discard:-edge_discard], v[edge_discard:-edge_discard])
        # plt.plot(sorted(v_clip))
        plt.plot(v_clip)
        plt.plot(max_influence, 'm')
        #plt.plot(v_max_rivaling, 'c')
        # plt.plot(np.full_like(v_clip, v_mean + v_std*3), 'r')
        # plt.plot(np.full_like(v_clip, v_mean),           'g')
        # plt.plot(np.full_like(v_clip, v_mean - v_std*3), 'r')
        plt.plot(np.full_like(v_clip, v_20),     'c')
        plt.plot(np.full_like(v_clip, v_median), 'g')
        plt.plot(np.full_like(v_clip, v_80),     'c')
        #plt.plot([v[0] for v in v_peaks_not_near], [v[1] for v in v_peaks_not_near], 'k.')
        plt.savefig("conf-" + os.path.split(f)[1][12:-4] + ".png")
        plt.close()

for v in sorted(conv_results, key=lambda v: v[1]):
    print(f'{v[0]:50s}: {v[1]*100:0.2f}% confidence')
